In [ ]:
!export CUDA_VISIBLE_DEVICES=3

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
import pdb

In [ ]:
hyp = {
    'dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'valid_data': '/mnt/data/Speech/librispeech_tse/dev-clean-mix/seed123/meta.csv',
    'base_folder_dm': '/mnt/nvme/Speech/librispeech/train-clean-100/train-clean-100/',
    'base_folder_dm_ext': 'flac',
    'sample_rate': 8000,
    'training_signal_len': 24000,
}

In [ ]:
from data_mixing import static_mixing_prep
import torch
import numpy as np
from speechbrain.dataio.batch import PaddedBatch

# Test static mixing 

In [ ]:


ds = static_mixing_prep(hyp, 'valid')
dl = torch.utils.data.DataLoader(
        ds,
        batch_size=hyp["dataloader_opts"]["batch_size"],
        num_workers=hyp["dataloader_opts"]["num_workers"],
        collate_fn=PaddedBatch,
        worker_init_fn=lambda x: np.random.seed(
            int.from_bytes(os.urandom(4), "little") + x
        ),
    )

In [ ]:
def listen_batch(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    diff = mix - s1 - s2
    print(abs(mix).max())
    print((diff**2).mean())
    sound(mix[0].numpy(), rate=hyp['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hyp['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hyp['sample_rate'], label='s2')
    sound(enr[0].numpy(), rate=hyp['sample_rate'], label='enr')

In [ ]:
for i, batch in enumerate(dl):  
    if i == 5:
        listen_batch(batch)
        break

# Test dynamic mixing 

In [ ]:
hyp = {
    'dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'base_folder_dm_info_list': [
        {
            'path': '/mnt/data/wsj0.8k/si_tr_s/',
            'ext': 'wav',
            'type': 'clean',
        },
        {
            'path': '/mnt/data/Speech/VoxCeleb1/dev_wav/',
            'ext': 'wav',
            'type': 'unclean',
        }
    ],
    'train_data': '/mnt/data/zhepei/outputs/sb_tse/results/2021-09-07+10-05-10+seed_1234+sepformer_wsj_emb/save/dummy.csv',
    'data_clean_prob': 0.5,
    'sample_rate': 8000,
    'training_signal_len': 24000,
}

from data_mixing import dynamic_mixing_prep
dl = dynamic_mixing_prep(hyp, 'train')

In [ ]:
def listen_batch_dyn(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
#     diff = mix - s1 - s2
#     print(abs(mix).max())
#     print((diff**2).mean())
    print(batch['s1_clean'].data)
    print(batch['s2_clean'].data)
    sound(mix[0].numpy(), rate=hyp['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hyp['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hyp['sample_rate'], label='s2')
    sound(enr[0].numpy(), rate=hyp['sample_rate'], label='enr')

In [ ]:
for i, batch in enumerate(dl):  
    if i == 10:
        listen_batch_dyn(batch)
        break

# Sepformer dataloader 

In [ ]:
from hyperpyyaml import load_hyperpyyaml
with open('/home/zhepei/workspace/playground/sb/separation/hparams/sepformer.yaml') as fin:
    hparams = load_hyperpyyaml(fin)

In [ ]:
from sepformer_mix import dynamic_mix_data_prep
dl = dynamic_mix_data_prep(hparams)

In [ ]:
def listen_batch_wsj(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    diff = mix - s1 - s2
    print(abs(mix).max())
    print((diff**2).mean())
    sound(mix[0].numpy(), rate=hyp['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hyp['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hyp['sample_rate'], label='s2')

    
for i, batch in enumerate(dl):  
    if i == 0:
        listen_batch_wsj(batch)
        break